In [ ]:
import json
with open('TEST.json', 'r', encoding='utf-8') as file:
    box = json.load(file)
    skip_project_name = None
    task_to_update = []
    new_task_after_reset = []
    all_task = box.get('tasks')
    for task in all_task:
        id_in_work_composer = task.get('_id')
        id_projects = task.get('_project')
        id_users = task.get('_assignedTo')
        title = task.get('title')
        description = task.get('description')
        author = task.get('_author')
        status = task.get('status')

        if skip_project_name is not None and '[Сайты]' in title and skip_project_name in title:
            continue

        data = {
            'id_in_work_composer': id_in_work_composer,
            'id_projects': id_projects,
            'id_users': id_users,
            'title': title,
            'description': description,
            'author': author,
            'status': status,
            'id_project_site': ''
        }
        if project_site is not None and '[Сайты]' in title and project_site.project in title:
            data['id_project_site'] = project_site.id

        new_task = await get_tasks_in_db(id_in_work_composer)
        if new_task is not None:
            if_change_task = [
                new_task.status != status,
                new_task.id_users != id_users,
                new_task.title != title,
                new_task.description != description
            ]
            if any(if_change_task):
                logger_to_project.info(f'{new_task.title} is update with {data = }')
                new_update_task = await update_task(id_in_work_composer, data)
                if new_update_task:
                    # if new_task.id_project_site != '':
                    #     skip_project_name = await update_google_sheets_from_work_composer(
                    #         new_task,
                    #         id_projects,
                    #         description,
                    #         id_users,
                    #         data
                    #     )
                    task_to_update.append((new_task, data))
        else:
            new_task = await create_tasks(data)
            if if_reset is None and new_task:
                new_task_after_reset.append((new_task, tags))
                logger_to_project.info(f'new_task {new_task}')
                continue

In [6]:
from settings.db import AsyncSessionLocal, ChatId, Projects, ProjectsSite, Tasks, Users
from sqlalchemy import select
import asyncio

In [3]:
async def get_tasks_in_db(tasks_id_in_work_composer):
    async with AsyncSessionLocal() as session:
        try:
            db_task_id = await session.execute(
                select(Tasks).where(
                    Tasks.id_in_work_composer == tasks_id_in_work_composer
                )
            )
            db_task_id = db_task_id.scalars().first()
        except Exception:
            db_task_id = None
    return db_task_id

In [7]:
x = '652e370c7967ae40e84755c7'
#f = get_tasks_in_db(x)
f = asyncio.run(get_tasks_in_db(x))

RuntimeError: asyncio.run() cannot be called from a running event loop